<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/PSO_vs_Alpine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Función Alpine
Empecemos viendo la regla de correspondencia y gráfica de la función Alpine.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def alpine(x, y):
    """Calcula el valor de la funcion Alpine."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

# Optimización por enjambre de partículas (PSO)
Veamos el comportamiento de PSO con la función alpine.

In [ ]:
def pso(max_iter, n_particulas, w, c1, c2, rango_x=(-5, 5), rango_y=(-5, 5)):
    """Optimiza la funcion Alpine usando PSO."""

    # Inicializar posiciones aleatorias en el rango especificado
    particulas = [
        (random.uniform(rango_x[0], rango_x[1]),
         random.uniform(rango_y[0], rango_y[1]))
        for _ in range(n_particulas)
    ]

    # Inicializar velocidades pequenas
    velocidades = [
        (random.uniform(-1, 1), random.uniform(-1, 1))
        for _ in range(n_particulas)
    ]

    # Evaluar funcion objetivo para cada particula
    valores = [alpine(x, y) for x, y in particulas]

    # Mejor posicion personal (pbest): inicialmente es la posicion actual
    pbest = particulas[:]  # Copia de las posiciones
    pbest_valores = valores[:]  # Copia de los valores

    # Mejor posicion global (gbest): la mejor entre todas las particulas
    indice_mejor = np.argmin(valores)
    gbest = particulas[indice_mejor]
    gbest_valor = valores[indice_mejor]

    # Guardar historial para graficar convergencia
    historial_valores = [gbest_valor]

    # Guardar coordenadas de todas las particulas en cada iteracion
    coordenadas_x = [np.array([p[0] for p in particulas])]
    coordenadas_y = [np.array([p[1] for p in particulas])]

    for iteracion in range(max_iter):
        for i in range(n_particulas):
            # Posicion y velocidad actual de la particula i
            x, y = particulas[i]
            vx, vy = velocidades[i]

            # Generar numeros aleatorios para estocasticidad
            r1 = random.random()
            r2 = random.random()

            # Actualizar velocidad segun ecuacion de PSO
            vx_nueva = (w * vx +
                        c1 * r1 * (pbest[i][0] - x) +
                        c2 * r2 * (gbest[0] - x))

            vy_nueva = (w * vy +
                        c1 * r1 * (pbest[i][1] - y) +
                        c2 * r2 * (gbest[1] - y))

            # Actualizar posicion: nueva_posicion = posicion + velocidad
            x_nueva = x + vx_nueva
            y_nueva = y + vy_nueva

            # Asegurar que la posicion este dentro del rango permitido
            x_nueva = np.clip(x_nueva, rango_x[0], rango_x[1])
            y_nueva = np.clip(y_nueva, rango_y[0], rango_y[1])

            # Guardar nueva posicion y velocidad
            particulas[i] = (x_nueva, y_nueva)
            velocidades[i] = (vx_nueva, vy_nueva)

            # Evaluar funcion objetivo en la nueva posicion
            valor_nuevo = alpine(x_nueva, y_nueva)
            valores[i] = valor_nuevo

            # Actualizar pbest si mejoramos la mejor posicion personal
            if valor_nuevo < pbest_valores[i]:
                pbest[i] = (x_nueva, y_nueva)
                pbest_valores[i] = valor_nuevo

            # Actualizar gbest si mejoramos la mejor posicion global
            if valor_nuevo < gbest_valor:
                gbest = (x_nueva, y_nueva)
                gbest_valor = valor_nuevo

            # Guardar el mejor valor de esta iteracion (para grafica de convergencia)
        historial_valores.append(gbest_valor)

        # Guardar coordenadas de todas las particulas (para animacion)
        coordenadas_x.append(np.array([p[0] for p in particulas]))
        coordenadas_y.append(np.array([p[1] for p in particulas]))

    # Retornar: mejor solucion, historial y coordenadas
    return (gbest[0], gbest[1], gbest_valor,
            historial_valores, coordenadas_x, coordenadas_y)

## Configuración de parámetros y ejecución del algoritmo.

In [ ]:
# === Configuracion de parametros ===
max_iter = 20
n_particulas = 5
w = 0.75   # Inercia
c1 = 0.50  # Coeficiente cognitivo
c2 = 0.50  # Coeficiente social

# === Ejecutar PSO ===
x_opt, y_opt, valor_opt, historial, coord_x, coord_y = pso(
    max_iter=max_iter,
    n_particulas=n_particulas,
    w=w,
    c1=c1,
    c2=c2
)

# === Imprimir resultados ===
print("Mejor solucion encontrada:")
print(f"  X = {x_opt:.6f}")
print(f"  Y = {y_opt:.6f}")
print(f"  Valor de Alpine = {valor_opt:.6f}")
print(f"  Optimo teorico: f(0, 0) = 0")

# Veamos una gráfica de convergencia
La gráfica muestra cómo el valor de $gbest$ disminuye (mejora) a lo largo de las iteraciones. Típicamente, verás una mejora rápida al inicio y luego una convergencia lenta hacia el óptimo.

In [ ]:
# === Crear grafica de convergencia ===
plt.figure(figsize=(10, 6))
iteraciones = range(len(historial))

plt.plot(iteraciones, historial, 'b-', linewidth=2, label='Mejor valor')
plt.scatter(iteraciones, historial, c='red', s=30, zorder=5)

plt.xlabel('Iteracion', fontsize=12)
plt.ylabel('Mejor valor encontrado (gbest)', fontsize=12)
plt.title('Convergencia de PSO en funcion Alpine', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Veamos la trayectoria de una partícula.
Dado que en PSO se emplean múltiples soluciones simultáneamente, resulta complicado representar su movimiento en una sola imagen. Por lo tanto, únicamente se mostrará la posición una partícula a medida que avanza el algoritmo.

In [ ]:
# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura ===
plt.figure(figsize=(10, 8))

# Graficar curvas de nivel
plt.contour(X, Y, Z, levels=20, colors='gray', alpha=0.5)
plt.contourf(X, Y, Z, levels=30, cmap='viridis', alpha=0.6)

# === Seleccionar particula a visualizar ===
particula_id = 4

# === Graficar trayectoria de la particula ===
plt.plot(
    coord_x[particula_id],
    coord_y[particula_id],
    color='red',
    marker='o',
    markersize=6,
    linestyle='-',
    linewidth=2,
    label=f'Partícula {particula_id}'
)

# === Agregar flechas para mostrar direccion ===
for i in range(1, len(coord_x[particula_id]) - 1):
    dx = (coord_x[particula_id][i] - coord_x[particula_id][i-1]) * 0.5
    dy = (coord_y[particula_id][i] - coord_y[particula_id][i-1]) * 0.5

    plt.arrow(
        coord_x[particula_id][i-1],
        coord_y[particula_id][i-1],
        dx, dy,
        color='black',
        head_width=0.1,
        head_length=0.1,
        zorder=10
    )

plt.colorbar(label='Valor de Alpine')
plt.xlabel('X', fontsize=12)
plt.ylabel('Y', fontsize=12)
plt.title(f'Trayectoria de la partícula {particula_id}',
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#Veamos una animación

In [ ]:
import matplotlib.animation as animation
from IPython.display import Video  # Para mostrar el video en Colab

In [ ]:
import matplotlib.animation as animation
from IPython.display import Video  # Para visualizar en Colab

# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura y ejes ===
fig, ax = plt.subplots(figsize=(8, 7))

# Graficar curvas de nivel
contour = ax.contourf(X, Y, Z, levels=30, cmap='viridis')
ax.contour(X, Y, Z, levels=20, colors='black', alpha=0.3)

# Agregar barra de color
plt.colorbar(contour, ax=ax, label='Valor de Alpine')

# Configurar limites y etiquetas
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('Movimiento del enjambre en PSO', fontsize=14, fontweight='bold')

# === Inicializar circulos para cada particula ===
circles = [
    plt.Circle((coord_x[0][i], coord_y[0][i]), 0.15,
               color='red', zorder=5)
    for i in range(n_particulas)
]

# Agregar circulos al grafico
for circle in circles:
    ax.add_artist(circle)

# === Funcion de actualizacion para cada frame ===
def actualizar_frame(frame):
    """Actualiza la posicion de cada particula en el frame dado."""
    for i, circle in enumerate(circles):
        # Actualizar centro del circulo a la nueva posicion
        nueva_posicion = (coord_x[frame][i], coord_y[frame][i])
        circle.set_center(nueva_posicion)

    return circles

# === Crear animacion ===
animacion = animation.FuncAnimation(
    fig,
    actualizar_frame,
    frames=len(coord_x),
    interval=100,  # 100 ms entre frames
    blit=True,
    repeat=True
)

# === Guardar como archivo MP4 ===
animacion.save(
    'pso_animacion.mp4',
    writer='ffmpeg',
    fps=10,  # 10 frames por segundo
    dpi=100
)

print("Animacion guardada como: pso_animacion.mp4")

# === Visualizar en Google Colab ===
Video('pso_animacion.mp4', embed=True)